In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv('diabetic_data.csv')

In [18]:
pd.options.display.max_columns=155

In [2]:
data['readmitted']=data['readmitted'].replace('>30',0)
data['readmitted']=data['readmitted'].replace('NO',0)
data['readmitted']=data['readmitted'].replace('<30',1)
data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,0
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,0
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,0


In [3]:
x=data.drop('readmitted',axis=1)
y=data.readmitted

In [4]:
x.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,No
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,No,Yes
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,No,Steady,No,No,No,No,No,Ch,Yes


In [5]:
x.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed'],
      dtype='object')

In [6]:
def first_letter(col):
    if (col[0] == 'E' or col[0] == 'V'):
        return '7777'
    else:
        return col
def cat_col(col):
    if (col >= 390) & (col <= 459) | (col == 785):
        return 'circulatory'
    elif (col >= 460) & (col <= 519) | (col == 786):
        return 'respiratory'
    elif (col >= 520) & (col <= 579) | (col == 787):
        return 'digestive'
    elif (col >= 250.00) & (col <= 250.99):
        return 'diabetes'
    elif (col >= 800) & (col <= 999):
        return 'injury'
    elif (col >= 710) & (col <= 739): 
        return 'musculoskeletal'
    elif (col >= 580) & (col <= 629) | (col == 788):
        return 'genitourinary'
    elif ((col >= 290) & (col <= 319) | (col == 7777) | 
          (col >= 280) & (col <= 289) | 
          (col >= 320) & (col <= 359) |
          (col >= 630) & (col <= 679) |
          (col >= 360) & (col <= 389) |
          (col >= 740) & (col <= 759)):
        return 'other'
    else:
        return 'neoplasms' 
    
def binary_lab_procedures(col):
    if (col >= 1) & (col <= 10):
        return '[1-10]'
    if (col >= 11) & (col <= 20):
        return '[11-20]'
    if (col >= 21) & (col <= 30):
        return '[21-30]'
    if (col >= 31) & (col <= 40):
        return '[31-40]'
    if (col >= 41) & (col <= 50):
        return '[41-50]'
    if (col >= 51) & (col <= 60):
        return '[51-60]'
    if (col >= 61) & (col <= 70):
        return '[61-70]'
    if (col >= 71) & (col <= 80):
        return '[71-80]'
    if (col >= 81) & (col <= 90):
        return '[81-90]'
    if (col >= 91) & (col <= 100):
        return '[91-100]'
    if (col >= 101) & (col <= 110):
        return '[101-110]'
    if (col >= 111) & (col <= 120):
        return '[111-120]'
    else:
        return '[121-132]' 

In [7]:
# data=x

In [8]:
def preprocess(data):
#     data.reset_index(inplace=True)
    data.rename(columns = {'time_in_hospital':'no_of_days_admitted'},inplace=True)
    data['num_visits'] = data.groupby('patient_nbr')['patient_nbr'].transform('count')
    data.sort_values(by = 'patient_nbr', ascending = True,inplace=True)
    data.sort_values(['patient_nbr', 'encounter_id'],inplace=True)
    data.drop_duplicates(['patient_nbr'],inplace=True)
    data=data[((data.discharge_disposition_id != 11) & 
                                          (data.discharge_disposition_id != 13) &
                                          (data.discharge_disposition_id != 14) & 
                                          (data.discharge_disposition_id != 19) & 
                                          (data.discharge_disposition_id != 20) & 
                                          (data.discharge_disposition_id != 21))]
    data.groupby('discharge_disposition_id').size()
    data = data[((data.race != '?'))]
    data.replace(to_replace='?', value=np.nan, inplace=True)
    data = data.drop(['weight', 'medical_specialty', 'payer_code'], axis = 1)
    data = data[((data.diag_1 != '?') &
                                (data.diag_2 != '?') &
                                (data.diag_3 != '?'))]
    d1 = pd.DataFrame(data.diag_1.apply(lambda col: first_letter(str(col))), dtype = 'float')
    d2 = pd.DataFrame(data.diag_2.apply(lambda col: first_letter(str(col))), dtype = 'float')
    d3 = pd.DataFrame(data.diag_3.apply(lambda col: first_letter(str(col))), dtype = 'float')
    data = pd.concat([data, d1, d2, d3], axis = 1)
    data.columns.values[47:50] = ('Diag1', 'Diag2', 'Diag3')
    data = data.drop(['diag_1', 'diag_2', 'diag_3'], axis = 1)
    data['first_diag'] = data.Diag1.apply(lambda col: cat_col(col))
    data['second_diag'] = data.Diag2.apply(lambda col: cat_col(col))
    data['third_diag'] = data.Diag3.apply(lambda col: cat_col(col))
    data.rename(columns={'glyburide-metformin': 'glyburide_metformin',
                       'glipizide-metformin': 'glipizide_metformin',
                       'glimepiride-pioglitazone': 'glimepiride_pioglitazone',
                       'metformin-rosiglitazone': 'metformin_rosiglitazone',
                       'metformin-pioglitazone': 'metformin_pioglitazone', }, inplace=True)
    data = data.drop(['encounter_id', 'patient_nbr', 'Diag1', 'Diag2', 'Diag3'], axis = 1)
    data = data.replace('?', np.NaN)
    data.loc[(data.gender == 'Unknown/Invalid'),'gender']='Female'    
    data['HbA1c'] = np.where(data['A1Cresult'] == 'None', 0, 1)
    data['num_lab_procedure_ranges'] = data['num_lab_procedures'].apply(lambda x: binary_lab_procedures(x))
    data=data.drop(['num_lab_procedures'], axis = 1)
    columns = data[['admission_type_id', 'discharge_disposition_id', 'admission_source_id']] 
    data[['admission_type_id', 'discharge_disposition_id', 'admission_source_id']] = columns.astype(object)
    data_encoded=data.apply(LabelEncoder().fit_transform)
    return data_encoded



In [9]:
data=pd.DataFrame(x.iloc[1000:2000])

In [10]:
# best_model = joblib.load('stacking_classifier_model_final.pkl')
data_encoded=preprocess(data)
# label = best_model.predict(data_encoded)
# print('The label is : ', label)

In [11]:
data_encoded.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,no_of_days_admitted,num_procedures,num_medications,number_outpatient,...,metformin_rosiglitazone,metformin_pioglitazone,change,diabetesMed,num_visits,first_diag,second_diag,third_diag,HbA1c,num_lab_procedure_ranges
1164,0,0,5,0,0,6,5,2,12,0,...,0,0,1,1,0,7,1,6,0,5
1083,2,1,6,0,0,6,1,0,13,0,...,0,0,1,1,0,6,0,2,0,2
1309,2,0,7,0,4,6,5,0,21,0,...,0,0,0,1,0,0,0,0,1,8
1932,2,1,6,0,5,6,2,0,9,0,...,0,0,1,1,0,4,1,6,0,5
1020,2,1,6,2,0,0,4,0,18,0,...,0,0,0,1,0,3,0,4,0,5


In [15]:
def fun1(data):
    best_model = joblib.load('stacking_classifier_model_final_last.pkl')
    data_encoded=preprocess(data)
    label = best_model.predict(data_encoded)
    print('The label is : ', label)
    

In [19]:
fun1(x.iloc[1000:2000])

The label is :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 

In [62]:
from sklearn.metrics import *
def function_2(data, label):
    '''This function will calculate metric for given input features'''
    best_model = joblib.load('stacking_classifier_model_final_last.pkl')
    data_encoded=preprocess(data)
    pred_label = best_model.predict(data_encoded)
    label=label[0:len(data_encoded)]
    acc=accuracy_score(label, pred_label)
    f1_score_ = f1_score(label, pred_label, average='macro')
    print('f1-score is : ', f1_score_)
    print("Accuracy is :", acc)

In [63]:
function_2(x.iloc[100:200],y.iloc[100:200])

f1-score is :  0.4789473684210527
Accuracy is : 0.9191919191919192
